In [1]:
import pandas as pd

In [2]:
prod_15 = pd.read_csv("../data/products_over15.csv")
order_prod_prior = pd.read_csv("../data/order_products__prior.csv")
order_prod_train = pd.read_csv("../data/order_products__train.csv")
orders = pd.read_csv("../data/orders.csv")

In [3]:
#prod_15.head()

In [4]:
#order_prod_prior.head()

In [5]:
#order_prod_train.head()

In [6]:
#orders.head()

In [4]:
#len(order_prod_train.order_id.unique())

In [5]:
#order_prod_train.shape

In [6]:
#len(order_prod_prior.order_id.unique())

In [7]:
#order_prod_prior.shape

In [8]:
#order_prod_prior[order_prod_prior["order_id"] == 6]

In [4]:
order_prod = order_prod_prior.append(order_prod_train, ignore_index=True)

In [70]:
#order_prod.head()

order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0

In [5]:
#order_prod['rating'] = order_prod['reordered'] + 1
order_prod = order_prod.drop(["add_to_cart_order", "reordered"], axis = 1)
order_prod.head()

order_id  product_id
0         2       33120
1         2       28985
2         2        9327
3         2       45918
4         2       30035

In [54]:
#orders.head()

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0

In [60]:
#len(orders["user_id"].unique())

206209

In [6]:
order_user = orders[['order_id', 'user_id']]
#print(order_user.head())
#print(order_prod.head())

In [73]:
# orders and products by order 
#print(order_prod.shape)
# order id - user id mapping 
#print(order_user.shape)

(33819106, 2)
(3421083, 2)


In [7]:
user_prod = pd.merge(order_prod, order_user, on='order_id', how='left').drop('order_id', axis = 1)
user_prod.head()

product_id  user_id
0       33120   202279
1       28985   202279
2        9327   202279
3       45918   202279
4       30035   202279

In [77]:
#user_prod.shape

(33819106, 2)

In [8]:
user_prod['count'] = 1
user_prod.head()

product_id  user_id  count
0       33120   202279      1
1       28985   202279      1
2        9327   202279      1
3       45918   202279      1
4       30035   202279      1

In [9]:
# customer clusters 
cluster0 = pd.read_pickle('cluster_0')
cluster1 = pd.read_pickle('cluster_1')
cluster2 = pd.read_pickle('cluster_2')

In [10]:
#user_prod.groupby(["product_id", "user_id"]).count()
data = user_prod.groupby(["product_id", "user_id"])["count"].count().reset_index(name="count")

In [131]:
data = data[['user_id','product_id', 'count']]
#data

user_id  product_id  count
0             138           1      2
1             709           1      1
2             751           1      1
3             764           1      2
4             777           1      1
...           ...         ...    ...
13863741   197371       49688      1
13863742   200215       49688      1
13863743   200377       49688      1
13863744   200873       49688      2
13863745   205926       49688      1

[13863746 rows x 3 columns]

In [11]:
samp_data = data.sample(frac=0.05, replace=False, random_state=1)

In [94]:
#print(len(cluster0))
#print(len(cluster1))
#print(len(cluster2))

24703
43035
138471


In [12]:
# subset user-product into 3 subsets
# with customer clusters 
data0 = data[data['user_id'].isin(cluster0)]
data1 = data[data['user_id'].isin(cluster1)]
data2 = data[data['user_id'].isin(cluster2)]

now data0, 1, 2 are preprocessed into format the surprise library takes.

plan: run different recommender systems on these subsets of the data to see if performance changes from running it on all customers

In [104]:
#conda install -c conda-forge scikit-surprise

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-surprise-1.1.1      |   py38h174b24a_1         586 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         586 KB

The following NEW packages will be INSTALLED:

  scikit-surprise    conda-forge/osx-64::scikit-surprise-1.1.1-py38h174b24a_1



scikit-surprise-1.1. | 586 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


#### General approach

FILL IN

In [13]:
from surprise import Dataset
from surprise import Reader

In [14]:
reader = Reader(rating_scale=(1, 101))
data = Dataset.load_from_df(data[['user_id', 'product_id', 'count']], reader)
data0 = Dataset.load_from_df(data0[['user_id', 'product_id', 'count']], reader)
data1 = Dataset.load_from_df(data1[['user_id', 'product_id', 'count']], reader)
data2 = Dataset.load_from_df(data2[['user_id', 'product_id', 'count']], reader)
samp_data = Dataset.load_from_df(samp_data[['user_id', 'product_id', 'count']], reader)

In [15]:
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split


from surprise import SVDpp, NMF, SlopeOne, CoClustering, KNNBaseline, KNNBasic, KNNWithZScore, BaselineOnly

First we run SVD on 1) full data and then 2) the 3 segments (full broken into 3 subsets) to see if performance differs. 

In [23]:
# try CV on data0
# try CV, so results more robust
benchmark_0 = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), KNNBasic(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data0, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark_0.append(tmp) # changed to benchmark_0 after error msg
    
pd.DataFrame(benchmark_0).set_index('Algorithm').sort_values('test_rmse')    



Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


KeyError: "None of ['Algorithm'] are in the columns"

In [27]:
# typo from block above, forgot to change to benchmark_0 when append
# here benchmark is actually benchmark_0
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

test_rmse   fit_time  test_time
Algorithm                                    
BaselineOnly   1.093106   0.658078   2.811652
SVD            1.111859  11.503051   0.867332
SVDpp          1.123760  40.772415   2.609253
KNNBasic       1.178905  27.027613  18.394223
CoClustering   1.187577   6.708118   0.470420
SlopeOne       1.203635  15.992062   4.766140
NMF            1.204027  14.137813   3.620358
BaselineOnly   3.401873   1.646250   1.303675
SVD            3.448130  23.575058   2.433299
SVDpp          3.580704  50.176859   3.832049
NMF            3.697266  34.388143   5.054507
CoClustering   3.726586  19.216695   1.437688
SlopeOne       4.052783  35.033196   3.883378

In [29]:
# try CV on data1
# try CV, so results more robust
benchmark_1 = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), BaselineOnly(), CoClustering()]:
    print('here')
    # Perform cross validation
    results = cross_validate(algorithm, data1, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark_1.append(tmp)
    
pd.DataFrame(benchmark_1).set_index('Algorithm').sort_values('test_rmse')    




here
here
here
here
here
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
here


test_rmse     fit_time   test_time
Algorithm                                       
SlopeOne       4.659422   427.155067  492.065826
BaselineOnly   4.670651     7.742253   14.524952
CoClustering   4.763332    50.662264   19.170414
SVD            5.138838   171.908334   14.471877
NMF            5.368413   175.410924   19.289022
SVDpp         97.827278  3442.271826  158.484391

In [ ]:
# try CV on data1
# try CV, so results more robust
benchmark_2 = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), BaselineOnly(), CoClustering()]:
    print('here')
    # Perform cross validation
    results = cross_validate(algorithm, data2, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark_2.append(tmp)
    
pd.DataFrame(benchmark_2).set_index('Algorithm').sort_values('test_rmse')    




here
here


### grid search on sample data

In [18]:
# grid search for hyper parameter tuning 
# sample of full data
# baseline 
# ALS
from surprise.model_selection import GridSearchCV
print('Using ALS, set param_grid')
param_grid = {'bsl_options': {'method': ['als'],
                              'reg_i': [10, 1, 0.1, 0.01],
                              'reg_u': [15, 1.5],
                              'n_epochs':[5, 10]}
              }

print('grid search')
gs_bl = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

print('fitting BaselineOnly')
gs_bl.fit(samp_data)

# best RMSE score
print(gs_bl.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_bl.best_params['rmse'])

Using ALS, set param_grid
grid search
fitting BaselineOnly
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating 

In [19]:
# grid search for hyper parameter tuning 
# sample of full data
# baseline 
# SGD

print('Using SGD, set param_grid')
param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.2, 0.02, 0.002],
                              'learning_rate': [0.005, 0.05],
                              'n_epochs':[15, 20]}
              }

print('grid search')
gs_bl2 = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

print('fitting BaselineOnly')
gs_bl2.fit(samp_data)

# best RMSE score
print(gs_bl2.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_bl2.best_params['rmse'])

Using SGD, set param_grid
grid search
fitting BaselineOnly
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating 

In [21]:
# grid search for hyper parameter tuning 
# sample of full data
# SVD

from surprise.model_selection import GridSearchCV
print('Using SVD, set param_grid')
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}


print('grid search')
gs_svd = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

print('fitting SVD')
gs_svd.fit(samp_data)

# best RMSE score
print(gs_svd.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_svd.best_params['rmse'])

Using SVD, set param_grid
grid search
fitting SVD
3.43736663086827
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [25]:
# grid search for hyper parameter tuning 
# sample of full data
# SVD
from surprise.model_selection import GridSearchCV
print('Using SVD, set param_grid')
#param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#              'reg_all': [0.4, 0.6]}

param_grid = {'n_epochs': [10], 'lr_all': [0.005],
              'reg_all': [0.2, 0.02, 0.002]}

print('grid search')
gs_svd = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

print('fitting SVD')
gs_svd.fit(samp_data)

# best RMSE score
print(gs_svd.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_svd.best_params['rmse'])

Using SVD, set param_grid
grid search
fitting SVD
3.4362321104095597
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.2}


### grid search on segmented data

#### cluster 0:
baseline, SVD



In [22]:
# grid search for hyper parameter tuning 
# data0
# baseline 
# ALS
from surprise.model_selection import GridSearchCV
print('Using ALS, set param_grid')
param_grid = {'bsl_options': {'method': ['als'],
                              'reg_i': [10, 1, 0.1, 0.01],
                              'reg_u': [15, 1.5],
                              'n_epochs':[5, 10]}
              }

print('grid search')
gs_bl_c0 = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

print('fitting BaselineOnly')
gs_bl_c0.fit(data0)

# best RMSE score
print(gs_bl_c0.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_bl_c0.best_params['rmse'])

Using ALS, set param_grid
grid search
fitting BaselineOnly
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating 

In [24]:
# grid search for hyper parameter tuning 
# sample of full data
# baseline 
# SGD

print('Using SGD, set param_grid')
param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.2, 0.02, 0.002],
                              'learning_rate': [0.005, 0.05],
                              'n_epochs':[15, 20]}
              }

print('grid search')
gs_bl2_c0 = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

print('fitting BaselineOnly')
gs_bl2_c0.fit(data0)

# best RMSE score
print(gs_bl2_c0.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_bl2_c0.best_params['rmse'])

Using SGD, set param_grid
grid search
fitting BaselineOnly
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating 

In [26]:
# grid search for hyper parameter tuning 
# data0
# SVD

from surprise.model_selection import GridSearchCV
print('Using SVD, set param_grid')
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.2, 0.4, 0.6]}


print('grid search')
gs_svd_c0 = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

print('fitting SVD')
gs_svd_c0.fit(data0)

# best RMSE score
print(gs_svd_c0.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_svd_c0.best_params['rmse'])

Using SVD, set param_grid
grid search
fitting SVD
1.096993883898958
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.2}


In [37]:
# grid search for hyper parameter tuning 
# data0
# SVD

from surprise.model_selection import GridSearchCV
print('Using SVD, set param_grid')
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.2, 0.4, 0.6]}


print('grid search')
gs_svd_c0 = GridSearchCV(SVD, param_grid, measures=['mae'], cv=3)

print('fitting SVD')
gs_svd_c0.fit(data0)

# best RMSE score
print(gs_svd_c0.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs_svd_c0.best_params['mae'])

Using SVD, set param_grid
grid search
fitting SVD
0.691329426843334
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.2}


In [41]:
algo = gs_svd_c0.best_estimator['mae']
cross_validate(algo, data0, measures=['MAE'], cv=5, verbose=True)
#algo.fit(data0.build_full_trainset())

Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.6879  0.6828  0.6874  0.6870  0.6897  0.6870  0.0023  
Fit time          7.13    7.18    7.21    7.07    7.16    7.15    0.05    
Test time         1.13    0.39    0.38    0.33    1.03    0.65    0.35    


{'test_mae': array([0.68787733, 0.68277575, 0.68735596, 0.6870441 , 0.68972472]),
 'fit_time': (7.129241228103638,
  7.182843923568726,
  7.210119247436523,
  7.069298028945923,
  7.156614065170288),
 'test_time': (1.129824161529541,
  0.39392876625061035,
  0.3779029846191406,
  0.334578275680542,
  1.034945011138916)}

#### cluster 1:
baseline, coclustering


In [27]:
# grid search for hyper parameter tuning 
# data1
# baseline 
# ALS
from surprise.model_selection import GridSearchCV
print('Using ALS, set param_grid')
param_grid = {'bsl_options': {'method': ['als'],
                              'reg_i': [10, 1, 0.1, 0.01],
                              'reg_u': [15, 1.5],
                              'n_epochs':[5, 10]}
              }

print('grid search')
gs_bl_c1 = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

print('fitting BaselineOnly')
gs_bl_c1.fit(data1)

# best RMSE score
print(gs_bl_c1.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_bl_c1.best_params['rmse'])

Using ALS, set param_grid
grid search
fitting BaselineOnly
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating 

In [42]:
# grid search for hyper parameter tuning 
# data1
# baseline w/ MAE
# ALS
from surprise.model_selection import GridSearchCV
print('Using ALS, set param_grid')
param_grid = {'bsl_options': {'method': ['als'],
                              'reg_i': [10, 1, 0.1, 0.01],
                              'reg_u': [15, 1.5],
                              'n_epochs':[5, 10]}
              }

print('grid search')
gs_bl_c1 = GridSearchCV(BaselineOnly, param_grid, measures=['mae'], cv=3)

print('fitting BaselineOnly')
gs_bl_c1.fit(data1)

# best RMSE score
print(gs_bl_c1.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs_bl_c1.best_params['mae'])

Using ALS, set param_grid
grid search
fitting BaselineOnly
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating 

In [45]:
algo = gs_bl_c1.best_estimator['mae']
cross_validate(algo, data1, measures=['MAE'], cv=5, verbose=True)
#algo.fit(data0.build_full_trainset())

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     2.5691  2.5773  2.5671  2.5670  2.5615  2.5684  0.0051  
Fit time          9.07    10.17   9.35    10.17   9.69    9.69    0.44    
Test time         15.65   8.99    9.36    10.37   10.69   11.01   2.40    


{'test_mae': array([2.5690665 , 2.57729291, 2.56712279, 2.56699352, 2.56152226]),
 'fit_time': (9.071525812149048,
  10.16540265083313,
  9.348378896713257,
  10.173844814300537,
  9.689411878585815),
 'test_time': (15.649230003356934,
  8.986984014511108,
  9.357165098190308,
  10.365325927734375,
  10.689113140106201)}

In [28]:
# grid search for hyper parameter tuning 
# data1
# baseline 
# SGD

print('Using SGD, set param_grid')
param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.2, 0.02, 0.002],
                              'learning_rate': [0.005, 0.05],
                              'n_epochs':[15, 20]}
              }

print('grid search')
gs_bl2_c1 = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

print('fitting BaselineOnly')
gs_bl2_c1.fit(data1)

# best RMSE score
print(gs_bl2_c1.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_bl2_c1.best_params['rmse'])

Using SGD, set param_grid
grid search
fitting BaselineOnly
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating 

In [29]:
# grid search for hyper parameter tuning 
# data1
# SVD

from surprise.model_selection import GridSearchCV
print('Using SVD, set param_grid')
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.2, 0.4, 0.6]}


print('grid search')
gs_svd_c1 = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

print('fitting SVD')
gs_svd_c1.fit(data1)

# best RMSE score
print(gs_svd_c1.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_svd_c1.best_params['rmse'])

Using SVD, set param_grid
grid search
fitting SVD
4.6184231856959235
{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.2}


In [32]:
# grid search for hyper parameter tuning 
# data1
# CoClustering

from surprise.model_selection import GridSearchCV
print('Using CoClustering, set param_grid')
param_grid = {'n_cltr_u': [2,3,4], 'n_cltr_i': [2,3,4],
              'n_epochs': [15,20]}


print('grid search')
gs_cc_c1 = GridSearchCV(CoClustering, param_grid, measures=['rmse'], cv=3)

print('fitting CoClustering')
gs_cc_c1.fit(data1)

# best RMSE score
print(gs_cc_c1.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_cc_c1.best_params['rmse'])

Using CoClustering, set param_grid
grid search
fitting CoClustering
4.720553959511959
{'n_cltr_u': 3, 'n_cltr_i': 2, 'n_epochs': 20}


In [46]:
# grid search for hyper parameter tuning 
# data1
# CoClustering

from surprise.model_selection import GridSearchCV
print('Using CoClustering, set param_grid')
param_grid = {'n_cltr_u': [2,3,4], 'n_cltr_i': [2,3,4],
              'n_epochs': [15,20]}


print('grid search')
gs_cc_c1 = GridSearchCV(CoClustering, param_grid, measures=['mae'], cv=3)

print('fitting CoClustering')
gs_cc_c1.fit(data1)

# best RMSE score
print(gs_cc_c1.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs_cc_c1.best_params['mae'])

Using CoClustering, set param_grid
grid search
fitting CoClustering
2.5843522353754342
{'n_cltr_u': 2, 'n_cltr_i': 4, 'n_epochs': 15}


In [47]:
algo = gs_cc_c1.best_estimator['mae']
cross_validate(algo, data1, measures=['MAE'], cv=5, verbose=True)
#algo.fit(data0.build_full_trainset())

Evaluating MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     2.5711  2.5634  2.5720  2.5788  2.5786  2.5728  0.0057  
Fit time          63.34   59.24   61.63   60.84   63.93   61.79   1.70    
Test time         27.89   21.35   20.46   13.17   72.21   31.02   21.12   


{'test_mae': array([2.57107035, 2.56339981, 2.57204354, 2.57877365, 2.57859222]),
 'fit_time': (63.339065074920654,
  59.242690086364746,
  61.628377199172974,
  60.837337017059326,
  63.92515015602112),
 'test_time': (27.893249988555908,
  21.349342107772827,
  20.458837032318115,
  13.170156002044678,
  72.2122061252594)}

#### cluster2

baseline, 

In [33]:
# grid search for hyper parameter tuning 
# data2
# baseline 
# ALS
from surprise.model_selection import GridSearchCV
print('Using ALS, set param_grid')
param_grid = {'bsl_options': {'method': ['als'],
                              'reg_i': [10, 1, 0.1, 0.01],
                              'reg_u': [15, 1.5],
                              'n_epochs':[5, 10]}
              }

print('grid search')
gs_bl_c2 = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

print('fitting BaselineOnly')
gs_bl_c2.fit(data2)

# best RMSE score
print(gs_bl_c2.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_bl_c2.best_params['rmse'])

Using ALS, set param_grid
grid search
fitting BaselineOnly
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating 

In [34]:
# grid search for hyper parameter tuning 
# data1
# baseline 
# SGD

print('Using SGD, set param_grid')
param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.2, 0.02, 0.002],
                              'learning_rate': [0.005, 0.05],
                              'n_epochs':[15, 20]}
              }

print('grid search')
gs_bl2_c2 = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

print('fitting BaselineOnly')
gs_bl2_c2.fit(data2)

# best RMSE score
print(gs_bl2_c2.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_bl2_c2.best_params['rmse'])

Using SGD, set param_grid
grid search
fitting BaselineOnly
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating 

In [35]:
# grid search for hyper parameter tuning 
# data2
# SVD

from surprise.model_selection import GridSearchCV
print('Using SVD, set param_grid')
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.2, 0.4, 0.6]}


print('grid search')
gs_svd_c2 = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

print('fitting SVD')
gs_svd_c2.fit(data2)

# best RMSE score
print(gs_svd_c2.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs_svd_c2.best_params['rmse'])

Using SVD, set param_grid
grid search
fitting SVD
1.9845060382419426
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.2}
